**Installation and imports**

In [ ]:
!pip install tf2onnx

In [ ]:
import tensorflow as tf
import tf2onnx
import onnx

import boto3
from botocore.client import Config
from botocore.exceptions import ClientError
import datetime
import os

**Get experiment and run names from environment variables**

In [ ]:
experiment_name_param = os.getenv('experiment_name')
run_name_param = os.getenv('run_name')

**Convert model to onnx format**

In [ ]:
h5_model_path = "fracture_model.h5"
model = tf.keras.models.load_model(h5_model_path)

input_signature = [tf.TensorSpec([1, 256, 256, 3], tf.float32, name='x')]
onnx_model, _ = tf2onnx.convert.from_keras(model, input_signature, opset=13)

model_name = f"Classif-Exp-{experiment_name_param}-Run-{run_name_param}.onnx"

onnx.save(onnx_model, model_name)

**Retrieve MinIO credentials and initialize client**

In [ ]:
minio_endpoint = os.getenv('AWS_S3_ENDPOINT')
access_key = os.getenv('AWS_ACCESS_KEY_ID')
secret_key = os.getenv('AWS_SECRET_ACCESS_KEY')
bucket_name = "models"
region = os.getenv('AWS_DEFAULT_REGION')

s3_client = boto3.client(
    's3',
    endpoint_url=minio_endpoint,
    aws_access_key_id=access_key,
    aws_secret_access_key=secret_key,
    config=Config(signature_version='s3v4')
)

**Save model in MinIO object storage**

In [ ]:
try:
    response = s3_client.upload_file(model_name, 'models', f"fracture/{experiment_name_param}/{model_name}")
except ClientError as e:
    logging.error(e)